## import

In [2]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm
from time import sleep
import json, os


## instantiate webdriver

In [4]:
## instantiate driver
## check the version of Google Chrome and download correct version of chromedriver
driver = webdriver.Chrome()

In [18]:
## get page of "social grep", which gived old posts of subreddit
## e.g. https://socialgrep.com/search?query=%2Fr%2FLanguageTechnology%2Cafter%3A2010-01-01&order_by=oldest
## original reddit url = 'https://www.reddit.com/r/xxxxxxxxx/'

subreddit = 'Windows' # choose by yourself
start_date = '2010-01-01' # choose by yourself

url = f'https://socialgrep.com/search?query=%2Fr%2F{subreddit}%2Cafter%3A{start_date}&order_by=oldest'

driver.get(url)
repeat_time, waiting_time = 4, 2

## scroll to the bottom of the page and wait
for i in range(repeat_time):
    driver.execute_script(f"window.scrollTo(0, document.body.scrollHeight);")
    sleep(waiting_time)

In [19]:
## function to scrape
def get_content(post, subreddit):
    try:
        vote = int(post.select_one('span.text-info').text)
    except:
        vote = 0
    try:
        title = post.a.text
    except:
        return None
    try:
        text = post.select_one('div.post_content').get_text(separator='\n').strip()
        if text == '':
            text = None
    except:
        text = None
    date = post.select_one('h6.card-subtitle').text.split(',')[1].strip()

    if text == None and title == f"/r/{subreddit.lower()}":
        return None
    else:
        return {
            "vote" : vote,
            "title" : title,
            "text" : text,
            "date" : date
        }

## example of one post

In [20]:
soup = BeautifulSoup(driver.page_source)
posts = soup.select('div.card-body') # content is under here

get_content(posts[1], subreddit) # show one example

{'vote': 0,
 'title': "Does anyone know how to fix Vundo.JD? AVG 'fixes' it and it comes back alive again.",
 'text': None,
 'date': '2010-01-02'}

# for loop with datetime

In [67]:
if os.path.exists(f'{subreddit}.json'):
    ## resume scraping from the last date in the json file
    with open(f'{subreddit}.json', 'r', encoding='utf8') as f:
        scraped_data = json.load(f)
    new_date = scraped_data[-1]['date']
    url = f'https://socialgrep.com/search?query=%2Fr%2F{subreddit}%2Cafter%3A{new_date}&order_by=oldest'
else:
    ## if the file not exists, create a new list
    scraped_data = []

In [68]:
## scrape and append to `scraped_data`
## RUN THIS CELL AGAIN AND AGAIN until getting the latest post

for _ in tqdm(range(200)): # set repeat time 

    ## scroll to the bottom of the page and wait
    driver.get(url)
    for i in range(4):
        driver.execute_script(f"window.scrollTo(0, document.body.scrollHeight);")
        sleep(2)

    ## get HTML
    soup = BeautifulSoup(driver.page_source)
    posts = soup.select('div.card-body')

    ## iterate each post
    for post in posts:
        one_post_dict = get_content(post, subreddit)
        if one_post_dict != None:
            scraped_data.append(one_post_dict)

    ## save to json
    with open(f'{subreddit}.json', 'w', encoding='utf8') as f:
        json.dump(scraped_data, f, indent=False, ensure_ascii=False)

    ## set new date
    new_date = scraped_data[-1]['date']
    url = f'https://socialgrep.com/search?query=%2Fr%2F{subreddit}%2Cafter%3A{new_date}&order_by=oldest'


100%|██████████| 200/200 [58:20<00:00, 17.50s/it]  


## to dataframe and drop duplicate

In [17]:
df = pd.read_json(f'{subreddit}.json').drop_duplicates()
df

,vote,title,text,date
0,3,My Mac crashes when connecting USB Razer mouse,"Just a bug report, and a thread to help others...",2016-07-08
1,1,MacOS Sierra beta publique : télécharger et in...,None,2016-07-08
2,1,"[Problem] MacOS Sierra ""cannot be installed on...",[deleted],2016-07-08
3,0,Some questions regarding macOS Sierra public beta,Hi. If anyone has installed the public beta of...,2016-07-08
4,4,anybody know trustd's purpose?,"i used little snitch, good firewall program. s...",2016-07-08
...,...,...,...,...
19443,1,Should I update to Monterey,I have a MacBook Pro 15 inch early 2013 but it...,2022-01-05
19444,0,[deleted by user],[removed],2022-01-05
19445,1,[deleted by user],[removed],2022-01-05
19447,5,Why does text on certain pages of my book disa...,None,2022-01-05


In [21]:
## missing value in text
df.isna().sum()

vote        0
title       0
text     3265
date        0
dtype: int64

In [22]:
## text includes [removed] [deleted]
df[df['text'].isin(['[removed]', '[deleted]'])]

,vote,title,text,date
2,1,"[Problem] MacOS Sierra ""cannot be installed on...",[deleted],2016-07-08
10,6,"How is the battery life on public beta? Also, ...",[deleted],2016-07-09
15,3,Wifi calling on macOS,[deleted],2016-07-11
18,5,Seagate S Drive in Sierra beta,[deleted],2016-07-12
20,1,Any way to downgrade from macOS Sierra Public ...,[deleted],2016-07-13
...,...,...,...,...
19416,0,[deleted by user],[removed],2022-01-04
19420,1,iTunes automatically launching when connecting...,[removed],2022-01-04
19436,2,Which MacOS is better for older macs,[removed],2022-01-05
19444,0,[deleted by user],[removed],2022-01-05
